# Watson Speech to Textのカスタマイズ練習用Notebook

- [Watson Python SDK](https://github.com/watson-developer-cloud/python-sdk/blob/master/watson_developer_cloud/speech_to_text_v1.py)
- データ(MP3や用語辞書、トランスクリプト)も含んでいるので、資格情報だけ代えればこのままでも動きます
- ご自身のデータで実行するなら、データを準備の上、1.準備の 固定値のセルの値を変更してください

# 1.準備

## WatsonのPython SDKを未導入なら導入 [説明](https://github.com/watson-developer-cloud/python-sdk)

In [1]:
#!pip install --upgrade watson-developer-cloud
from watson_developer_cloud import SpeechToTextV1
import json

## IBM CloudのSTTサービスの資格情報(Credential)を丸ごとコピペ ( 使うのはapikeyだけ）
## ★STTのカスタマイズにはPAYGインスタンスが必要です。Liteプランではできません。

In [5]:
Credentials ={
  "apikey": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
  "iam_apikey_description": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
  "iam_apikey_name": "auto-generated-apikey-XXXXXXXXXXXXXXXXX",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/XXXXXXXXXXXXXXXXXXXXX",
  "url": "https://stream.watsonplatform.net/speech-to-text/api"
}

## インスタンス作成

In [6]:
stt = SpeechToTextV1(
    iam_apikey=Credentials["apikey"],
    url=Credentials["url"])

## 固定値のセット
### YouTubeの動画 [IBM Q - 量子コンピューター最新情報ならここ！](https://www.youtube.com/watch?v=-CJNJ1ETV5Q&t=33s)から抜いた音声(mp3)を利用

In [23]:
AUDIO_FILE = "mp3/IBMQ-01.mp3"
AUDIO_FORMAT = "audio/mp3"
BASE_MODEL = "ja-JP_BroadbandModel"
TRANSCRIPT_TEXT="Transcript.txt"   # コーパス
TRANSCRIPT_JSON="Transcript.json"  # 用語辞書
ACOUSTIC_FILE = "mp3/IBMQ-10min.mp3"

# 2.通常の（カスタマイズなし）認識

In [24]:
def recognizeNormal(audio_file):
    with open(audio_file,'rb') as audio_file:
        return  stt.recognize(
                      audio=audio_file,
                      content_type=AUDIO_FORMAT,
                      model=BASE_MODEL,
                      language_customization_id=None,
                      acoustic_customization_id=None,
                      base_model_version=None,
                      customization_weight=None,
                      inactivity_timeout=None,
                      keywords=None,
                      keywords_threshold=None,
                      max_alternatives=None,
                      word_alternatives_threshold=None,
                      word_confidence=None,
                      timestamps=None,
                      profanity_filter=None,
                      smart_formatting=None,
                      speaker_labels=None,
                      customization_id=None,
                      grammar_name=None,
                      redaction=None)
    

In [25]:
recognizeNormal(audio_file).get_result()

{'result_index': 0,
 'results': [{'alternatives': [{'confidence': 0.71,
     'transcript': 'ええ 、 役立た フロックス なん です けれども 並列 度 って いう 観点 から 見れば ええ ええ 一台 の 濃度 を 見て みる と D_エー 買わない の プロセッサー が 二つ D_エー それぞれ 二十 二個 D_アノー 四名 選定 の やつ プラス 一台 に ええ ええ ＣＴＵ エミリア 文字 で 言えば 六枚 です 刺さって ます で それぞれ D_エー 五千 百 二十個 と いう D_エー 凄い 並列 の です けれども ですから 一 ノード で 三万 を 超える 平熱 道 が あります '}],
   'final': True}]}

# 3.カスタム言語モデル

## 現状のカスタム言語モデルをリストする

In [27]:
# 整理のため、いったん全部削除する
#stt.delete_language_model(customization_id='5f99129a-79f7-4ffc-a58a-9551832cd3c3').get_result()
#stt.delete_language_model(customization_id='7ab0f0a3-74b4-4bd5-99d9-211a7319f77e').get_result()

In [28]:
stt.list_language_models(language=None).get_result()

{'customizations': []}

## カスタム言語モデルを作成する => IDを入手

In [29]:
result = stt.create_language_model(
      name="IBMQ_LanguageModel",
      base_model_name=BASE_MODEL,
      dialect=None,
      description="IBMQ_LanguageModel")

In [30]:
language_customization_id = result.get_result()["customization_id"]
print("language_customization_id:", language_customization_id)

language_customization_id: b5791dd4-f973-4ce2-885c-52228fc3549d


## カスタム言語モデルへのコーパスの追加

In [31]:
stt.add_corpus(
       customization_id=language_customization_id,
       corpus_name="IBMQ_Transcrpt",
       corpus_file= TRANSCRIPT_TEXT,
       allow_overwrite=True,
       corpus_filename=None).get_result()

{}

## カスタム言語モデルへの用語の追加（非同期処理)

In [32]:
with open(TRANSCRIPT_JSON,'r',encoding="utf-8") as f:
    custom_words = json.load(f)["words"]
    stt.add_words(
         customization_id=language_customization_id,
         words=custom_words).get_result()   

## 用語追加の完了状況をチェック( 'status': 'ready'になるまで）

In [33]:
stt.get_language_model(customization_id=language_customization_id).get_result()

{'base_model_name': 'ja-JP_BroadbandModel',
 'created': '2019-03-15T06:45:07.153Z',
 'customization_id': 'b5791dd4-f973-4ce2-885c-52228fc3549d',
 'description': 'IBMQ_LanguageModel',
 'dialect': 'ja-JP',
 'language': 'ja-JP',
 'name': 'IBMQ_LanguageModel',
 'owner': '2841151a-4baa-45c8-9618-5060d4d7beca',
 'progress': 0,
 'status': 'ready',
 'versions': ['ja-JP_BroadbandModel.v2018-03-08']}

## 登録されている用語の確認 

In [34]:
stt.list_words(customization_id=language_customization_id, word_type=None, sort=None).get_result()

{'words': [{'count': 1,
   'display_as': '200',
   'sounds_like': ['ニヒャク'],
   'source': ['user'],
   'word': '200'},
  {'count': 1,
   'display_as': 'GPU',
   'sounds_like': ['ジーピーユー'],
   'source': ['user'],
   'word': 'GPU'},
  {'count': 1,
   'display_as': 'NVIDIA',
   'sounds_like': ['エヌビディア'],
   'source': ['user'],
   'word': 'NVIDIA'},
  {'count': 1,
   'display_as': 'Power9',
   'sounds_like': ['パワーナイン'],
   'source': ['user'],
   'word': 'Power9'},
  {'count': 1,
   'display_as': 'SMT',
   'sounds_like': ['エスエムティ'],
   'source': ['user'],
   'word': 'SMT'},
  {'count': 1,
   'display_as': 'コア',
   'sounds_like': ['コア'],
   'source': ['user'],
   'word': 'コア'},
  {'count': 1,
   'display_as': 'ノード',
   'sounds_like': ['ノード'],
   'source': ['user'],
   'word': 'ノード'},
  {'count': 1,
   'display_as': 'ペタフロップス',
   'sounds_like': ['ペタフロップス'],
   'source': ['user'],
   'word': 'ペタフロップス'},
  {'count': 1,
   'display_as': '並列度',
   'sounds_like': ['ヘイレツ'],
   'source': ['user'],
   

In [35]:
# エラーがあれば個別に消す
#stt.delete_word(customization_id=language_customization_id, word_name='xxx').get_result()

## カスタム言語モデルのトレーニング実施

In [36]:
stt.train_language_model(
    customization_id=language_customization_id,
    word_type_to_add=None,
    customization_weight=None).get_result()

{}

## トレーニングステータスの確認（status=availableになるまで）

In [37]:
stt.get_language_model(language_customization_id).get_result()

{'base_model_name': 'ja-JP_BroadbandModel',
 'created': '2019-03-15T06:45:07.153Z',
 'customization_id': 'b5791dd4-f973-4ce2-885c-52228fc3549d',
 'description': 'IBMQ_LanguageModel',
 'dialect': 'ja-JP',
 'language': 'ja-JP',
 'name': 'IBMQ_LanguageModel',
 'owner': '2841151a-4baa-45c8-9618-5060d4d7beca',
 'progress': 100,
 'status': 'available',
 'versions': ['ja-JP_BroadbandModel.v2018-03-08']}

## カスタム言語モデルを使った音声to テキスト変換(STT)実行

In [38]:
def recognizeWithLangModel(audio_file):
    with open(audio_file,'rb') as audio_file:
        return stt.recognize(
                      audio=audio_file,
                      content_type= AUDIO_FORMAT,
                      model= BASE_MODEL,
                      language_customization_id=language_customization_id,
                      acoustic_customization_id=None,
                      base_model_version=None,
                      customization_weight=None,
                      inactivity_timeout=None,
                      keywords=None,
                      keywords_threshold=None,
                      max_alternatives=None,
                      word_alternatives_threshold=None,
                      word_confidence=None,
                      timestamps=None,
                      profanity_filter=None,
                      smart_formatting=None,
                      speaker_labels=None,
                      customization_id=None,
                      grammar_name=None,
                      redaction=None)

In [40]:
print(recognizeWithLangModel(AUDIO_FILE).result)

{'results': [{'alternatives': [{'confidence': 0.78, 'transcript': 'ええ 、 200 ペタフロップス なん です けれども 並列度 載ってる 観点 から 見れば ええ ええ 一台 の ノード を 見て みる と D_エー Power9 プロセッサー が 二つ D_エー それぞれ 二十 二 コア の 四 SMT の やつ プラス 一台 に ええ ええ GPU NVIDIA 文字 で 言えば 六枚 刺さって ます で それぞれ D_エー 五千 百 二十個 と いう D_エー 凄い 並列度 の です けれども ですから 一 ノード で 三万 を 超える 並列度 道 が あります '}], 'final': True}], 'result_index': 0}


# 4.カスタム音響モデル

## 音響モデルの作成

In [44]:
# 整理のため不要なものを消す
#stt.delete_acoustic_model(customization_id="213ca981-a3d7-4e59-824a-1dbabb4638ef").get_result()
#stt.delete_acoustic_model(customization_id="dd4315eb-51c0-44be-94bd-ea7bf4b47639").get_result()

In [43]:
stt.list_acoustic_models(language=None).get_result()

{'customizations': []}

In [45]:
result = stt.create_acoustic_model(
    name="IBMQ_AcousticModel",
    base_model_name= BASE_MODEL,
    description="IBMQ_AcousticModel")
result.get_result()

{'customization_id': 'f2a631c2-d90a-4d95-bd91-5d6ea512de79'}

In [47]:
acoustic_customization_id = result.get_result()["customization_id"]
print(acoustic_customization_id)

f2a631c2-d90a-4d95-bd91-5d6ea512de79


## 定義情報の確認

In [49]:
stt.get_acoustic_model(customization_id=acoustic_customization_id).get_result()

{'base_model_name': 'ja-JP_BroadbandModel',
 'created': '2019-03-15T07:01:57.753Z',
 'customization_id': 'f2a631c2-d90a-4d95-bd91-5d6ea512de79',
 'description': 'IBMQ_AcousticModel',
 'language': 'ja-JP',
 'name': 'IBMQ_AcousticModel',
 'owner': '2841151a-4baa-45c8-9618-5060d4d7beca',
 'progress': 0,
 'status': 'pending',
 'versions': ['ja-JP_BroadbandModel.v09-05202016.06202016']}

## 音声サンプルを登録(最低10分-MAX 100MB)　非同期

In [50]:
with open(ACOUSTIC_FILE,'rb') as sample_audio:
    stt.add_audio(
                  customization_id=acoustic_customization_id,
                  audio_name="IBM-Q-10min",
                  audio_resource=sample_audio,
                  content_type= AUDIO_FORMAT,
                  contained_content_type=None,
                  allow_overwrite=True).get_result()

## 登録の処理状況をチェック('status': 'ok'?)

In [51]:
stt.get_audio(customization_id=acoustic_customization_id, audio_name="IBM-Q-10min").get_result()

{'details': {'codec': 'mp3', 'frequency': 44100, 'type': 'audio'},
 'duration': 654,
 'name': 'IBM-Q-10min',
 'status': 'ok'}

## 音響モデルのトレーニングの実行(非同期実行）- カスタム言語モデルも併用

In [52]:
stt.train_acoustic_model(
    customization_id=acoustic_customization_id,
    custom_language_model_id=language_customization_id).get_result()

{}

##  トレーニングのステータスの確認 (status = available?)

In [62]:
stt.get_acoustic_model(customization_id=acoustic_customization_id).get_result() # check status

{'base_model_name': 'ja-JP_BroadbandModel',
 'created': '2019-03-15T07:01:57.753Z',
 'customization_id': 'f2a631c2-d90a-4d95-bd91-5d6ea512de79',
 'description': 'IBMQ_AcousticModel',
 'language': 'ja-JP',
 'name': 'IBMQ_AcousticModel',
 'owner': '2841151a-4baa-45c8-9618-5060d4d7beca',
 'progress': 100,
 'status': 'available',
 'versions': ['ja-JP_BroadbandModel.v09-05202016.06202016']}

## 音響モデルで認識を実行する

In [63]:
def reconnizeWithAcousticModel(audio_file):
    with open(audio_file,'rb') as audio_file:
        return  stt.recognize(
                      audio=audio_file,
                      content_type=AUDIO_FORMAT,
                      model=BASE_MODEL,
                      language_customization_id=language_customization_id,
                      acoustic_customization_id=acoustic_customization_id,
                      base_model_version=None,
                      customization_weight=None,
                      inactivity_timeout=None,
                      keywords=None,
                      keywords_threshold=None,
                      max_alternatives=None,
                      word_alternatives_threshold=None,
                      word_confidence=None,
                      timestamps=None,
                      profanity_filter=None,
                      smart_formatting=None,
                      speaker_labels=None,
                      customization_id=None,
                      grammar_name=None,
                      redaction=None)


In [64]:
print(reconnizeWithAcousticModel(AUDIO_FILE).result)

{'results': [{'alternatives': [{'confidence': 0.87, 'transcript': 'D_エー 200 ペタフロップス なん です けれども 並列度 載ってる 観点 から 見れば ええ ええ 一台 の ノード を 見て みる と D_エー Power9 プロセッサー が 二つ D_エー それぞれ 二十 二 コア の 四 SMT の やつ プラス 一台 に '}], 'final': True}, {'alternatives': [{'confidence': 0.91, 'transcript': 'GPU NVIDIA 文字 で 言えば 六枚 '}], 'final': True}, {'alternatives': [{'confidence': 0.88, 'transcript': '刺さって ます で それぞれ D_エー 五千 百 二十個 と いう D_エー 凄い 並列度 の です けれども ですから 一 ノード で 三万 を 超える 並列度 道 が あります '}], 'final': True}], 'result_index': 0}
